In [ ]:
# lets try 3000

In [ ]:
!pip install transformers==4.24.0
!pip install datasets rouge_score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=37781a66d28ff6cd95b4cd07d5c97968a14119e46a2afc27229a154747d52b63
  Stored in directory: /root/.cache/pip/whe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = "/content/drive/MyDrive/651_project/cnn_dailymail/train.csv"
test_path = "/content/drive/MyDrive/651_project/cnn_dailymail/test.csv"
validation_path = "/content/drive/MyDrive/651_project/cnn_dailymail/validation.csv"

In [ ]:
import transformers
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset("csv", data_files={"train": train_path, "test": test_path, "validation": validation_path})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 287113
    })
    test: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 11490
    })
    validation: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 13368
    })
})

In [ ]:
dataset["train"] = dataset["train"].shuffle().select(range(100000))
dataset["validation"] = dataset["validation"].shuffle().select(range(10000))
dataset["test"] = dataset["test"].shuffle().select(range(10000))

In [ ]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
dataset_cleaned = dataset.filter(
    lambda example: (len(example['article']) >= 700) and
    (len(example['highlights']) >= 60)
)

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
dataset_cleaned

DatasetDict({
    train: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 99568
    })
    test: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 9974
    })
    validation: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 9983
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
#model_name = "/content/drive/MyDrive/651_project/T5/t5-small/checkpoint-3000"
#model_name = "t5-small"
model_dir = f"/content/drive/MyDrive/651_project/T5/t5-small/checkpoint-3000"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)


In [ ]:
input_data = "By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained bishops in Italy last month. Symptoms of hepatitis A include fever, tiredness, loss of appetite, nausea and abdominal discomfort. Fargo Catholic Diocese in North Dakota (pictured) is where the bishop is located ."
inputs = ["summarize: " + input_data]
input


<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x7f4f40495ae0>>

In [ ]:
max_input_length = 512
max_target_length = 64

In [ ]:
max_input_length = 512
max_target_length = 64
inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)

Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members to the hepatitis A virus.


Prediction all test:

In [ ]:
metric = load_metric("rouge")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

<ipython-input-21-67e5721216c4>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
prefix = "summarize: "
max_input_length = 512
max_target_length = 64

def clean_text(text):
  sentences = nltk.sent_tokenize(text.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
  sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
                                 if len(sent) > 0 and
                                 sent[-1] in string.punctuation]
  text_cleaned = "\n".join(sentences_cleaned_no_titles)
  return text_cleaned

def preprocess_data(examples):
  texts_cleaned = [clean_text(text) for text in examples["article"]]
  inputs = [prefix + text for text in texts_cleaned]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["highlights"], max_length=max_target_length,
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_datasets = dataset_cleaned.map(preprocess_data,batched=True)

Map:   0%|          | 0/99568 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/9974 [00:00<?, ? examples/s]

Map:   0%|          | 0/9983 [00:00<?, ? examples/s]

In [ ]:
test_tokenized_dataset = tokenized_datasets["test"].shuffle().select(range(500))
test_tokenized_dataset

Dataset({
    features: ['id', 'article', 'highlights', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [ ]:
import torch
import numpy as np

def preprocess_test(examples):
  inputs = [prefix + text for text in examples["article"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding="max_length")
  return model_inputs

test_tokenized_dataset = test_tokenized_dataset.map(preprocess_test, batched=True)

test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=32)

all_predictions = []
for i,batch in enumerate(dataloader):
  predictions = model.generate(**batch)
  all_predictions.append(predictions)

all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

all_titles = tokenizer(test_tokenized_dataset["highlights"], max_length=max_target_length,
                       truncation=True, padding="max_length")["input_ids"]
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


500


{'rouge1': 27.1999,
 'rouge2': 13.4676,
 'rougeL': 22.6995,
 'rougeLsum': 25.4704,
 'gen_len': 19.0}

In [ ]:
import pandas as pd
test_path = "/content/drive/MyDrive/651_project/cnn_dailymail/test.csv"
df = pd.read_csv(test_path, nrows=500)

In [ ]:
df

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...,...
495,f8a8c9359369edf5fa1083979202141649725e01,Chelsea match-winner Cesc Fabregas hailed a vi...,Chelsea scored late winner to beat QPR 1-0 at ...
496,00f72087f0772310ab2926acc4a31102e51050ff,The brave man who stood up against a neo-Nazi ...,Jacob King has been identified as man who stoo...
497,8eb45776779a54e6f46da776598714065cb2be5c,A man strangled his friend to death in a rage ...,Steven Carl Day's confession over drinks that ...
498,3a8e2afdba04d7356526c94232f17eef9a5e4ddd,Reports of Cristiano Ronaldo's decline have be...,"Real Madrid beat Granada 9-1, with Cristiano R..."


In [ ]:
data_array = df["article"].to_numpy()

In [ ]:
max_input_length = 512
max_target_length = 64
pred = []
for i in data_array:
  inputs = ["summarize: " + i]
  inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
  output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]
  pred.append(predicted_title)
pred

['Some experts are questioning if shrinking space on planes is putting our health and safety in danger.',
 'Rahul Kumar, 17, climbed into enclosure fence at zoo in Ahmedabad.',
 'Dougie Freedman is set to sign a new two-year deal at Nottingham Forest.',
 'Liverpool target Neto is wanted by a number of top European clubs, according to his agent Stefano Castagna.',
 "The former Olympian and reality TV star, 65, will speak in a 'far-ranging' interview with Diane Sawyer later this month.",
 'The prize porker, known as Pigwig, had fallen into the pool in Ringwood, Hampshire.',
 'Average listener spent just ten hours a week tuning in to BBC radio.',
 '"Full House" will feature Candace Cameron Bure, who played eldest daughter D.J.',
 'Reanne Evans, 29, a single mum from Dudley, was a winner on Thursday night.',
 'Six men were jailed for a total of 31 years at Oxford Crown Court.',
 'Mohammed Ali Malek, 27, was arrested when he stepped onto Sicilian soil.',
 'Mark Lippert, 42, suffered deep ga